In [1]:
import pandas as pd
import numpy as np

import pymysql

#from datetime import date
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
host = '10.0.2.153'
port = int(3306)
dbname = 'remita_staging'
user = 'fiona'
password = 'fiona@2022'


In [3]:
def get_query_results():
    with pymysql.connect(host = host,
                          port = port,
                          database = dbname,
                          user = user,
                          password = password) as conn:
        sql = "SELECT * FROM SalaryHistory" 
        df = pd.read_sql_query(sql, conn)
        
    return df

In [4]:
salaries = get_query_results()

salaries.head(2)

C:\Users\FM\AppData\Local\Temp\ipykernel_9504\703973372.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


,Id,PhoneNo,BvnNo,CreatedDate,PaymentDate,Amount,AccountNumber,BankCode,CompanyName
0,1,2348130230131,2227134531,2022-06-10 11:01:23.422617,2022-02-25 13:33:46,155000.0,5012284010,023,None
1,2,2348130230131,2227134531,2022-06-10 11:01:23.422635,2022-01-25 13:33:46,155000.0,5012284010,023,None


In [6]:
## extracting dates from timestamp

salaries.loc[ : , 'payment_dates']  = [x.strftime('%Y-%m-%d') for x in salaries['PaymentDate']]

salaries['payment_dates'] = pd.to_datetime(salaries['payment_dates'])

salaries.loc[ : , 'year_month_salary_date'] = [x.strftime('%Y-%m') for x in salaries['PaymentDate']]

salaries.head(2)

,Id,PhoneNo,BvnNo,CreatedDate,PaymentDate,Amount,AccountNumber,BankCode,CompanyName,payment_dates,year_month_salary_date
0,1,2348130230131,2227134531,2022-06-10 11:01:23.422617,2022-02-25 13:33:46,155000.0,5012284010,023,None,2022-02-25,2022-02
1,2,2348130230131,2227134531,2022-06-10 11:01:23.422635,2022-01-25 13:33:46,155000.0,5012284010,023,None,2022-01-25,2022-01


In [7]:
## offset payment dates to the end of the month so that the filter that picks last 6 months of data does not pick 
## upto mid of a month as this would shortchange some applicants.

from pandas.tseries.offsets import MonthEnd

salaries['offset_payment_dates'] = pd.to_datetime(salaries['payment_dates'], format="%Y%m") + MonthEnd(0)

salaries.head()

,Id,PhoneNo,BvnNo,CreatedDate,PaymentDate,Amount,AccountNumber,BankCode,CompanyName,payment_dates,year_month_salary_date,offset_payment_dates
0,1,2348130230131,2227134531,2022-06-10 11:01:23.422617,2022-02-25 13:33:46,155000.0,5012284010,023,None,2022-02-25,2022-02,2022-02-28
1,2,2348130230131,2227134531,2022-06-10 11:01:23.422635,2022-01-25 13:33:46,155000.0,5012284010,023,None,2022-01-25,2022-01,2022-01-31
2,3,2348130230131,2227134531,2022-06-10 11:01:23.422637,2021-12-25 13:33:46,155000.0,5012284010,023,None,2021-12-25,2021-12,2021-12-31
3,4,23440971951,2220087024,2022-06-10 12:21:09.295481,2022-02-25 00:00:00,1074455.0,0783839390,044,None,2022-02-25,2022-02,2022-02-28
4,5,23440971951,2220087024,2022-06-10 12:21:09.295481,2022-01-25 00:00:00,1074455.0,0783839390,044,None,2022-01-25,2022-01,2022-01-31


In [8]:
## dropping duplicates

#salaries.drop_duplicates(subset = ['PhoneNo', 'BvnNo', 'PaymentDate', 'Amount','AccountNumber', 'BankCode'], inplace = True)

salaries.drop_duplicates(subset = ['BvnNo', 'PaymentDate', 'Amount','AccountNumber', 'BankCode'], inplace = True) 

## if bvn is tied to more than one number then only one number will remain after this executes.

In [9]:
max_date = salaries['PaymentDate'].max()

#max_date = pd.to_datetime('today')

print(max_date)

2023-07-03 18:59:06


In [10]:
max_month = pd.to_datetime(max_date.strftime('%Y-%m'))

max_month

Timestamp('2023-07-01 00:00:00')

In [12]:
max_month + relativedelta(months = -2)

Timestamp('2023-05-01 00:00:00')

In [14]:
def calculate_last_9_months_salaries(df):

    count = df['count_of_payments_last_9_months']

    if count >= 6 :
        return True
    else:
        return False


In [15]:
salaries_df = government_employees_salaries_check_3_and_6_month_product(salaries, max_month)

salaries_df.head(2)

/apps/conda/fiona.kamau/envs/tanda/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,BvnNo,count_of_payments_last_9_months,sum_of_salary_payments_last_9_months,minimum_salary_payment_last_9_months,earliest_salary_payment_date_last_9_months,latest_salary_payment_date_last_9_months,average_monthly_salary_last_9_months,latest_salary_payment_amount_last_9_months,is_qualified_on_salaries
0,22151967960,5,1481149.22,116322.79,2022-11-30,2023-03-31,296229.844,416509.94,False
1,22153417946,5,764935.17,3892.25,2022-11-25,2023-03-28,152987.034,68974.75,False


In [17]:
def government_employees_salaries_check_30_day_product(salaries, max_month):
    last_3_months = salaries[salaries['offset_payment_dates'] >= max_month + relativedelta(
        months=-2)]  # & (salaries['payment_dates'] < max_month)]

    monthly_salary_summaries = last_3_months.groupby(['BvnNo', 'year_month_salary_date'])['Amount'].sum().reset_index()

    monthly_salary_summaries = monthly_salary_summaries.groupby('BvnNo')['Amount'].mean().rename(
        'average_monthly_salary_last_3_months').reset_index()

    last_3_months_salaries_summaries = last_3_months.groupby(['BvnNo']).agg(
        count_of_payments_last_3_months=pd.NamedAgg('year_month_salary_date', 'nunique'),
        sum_of_salary_payments_last_3_months=pd.NamedAgg('Amount', 'sum'),
        minimum_salary_payment_last_3_months=pd.NamedAgg('Amount', 'min'),
        # median_salary_payment_last_6_months = pd.NamedAgg('Amount', 'median'),
        # mode_salary_payment_last_6_months = pd.NamedAgg('Amount', pd.Series.mode),
        earliest_salary_payment_date_last_3_months=pd.NamedAgg('payment_dates', 'min'),
        latest_salary_payment_date_last_3_months=pd.NamedAgg('payment_dates', 'max')
        ).reset_index()

    latest_payment_dates = last_3_months_salaries_summaries[['BvnNo', 'latest_salary_payment_date_last_3_months']]

    latest_payment_dates.loc[:, 'year_month_latest_salary_date'] = [x.strftime('%Y-%m') for x in latest_payment_dates[
        'latest_salary_payment_date_last_3_months']]

    last_3_months = last_3_months.merge(latest_payment_dates, on = 'BvnNo')

    latest_salary_payment_amount_last_3_months = last_3_months[last_3_months['year_month_latest_salary_date'] == last_3_months['year_month_salary_date']].groupby(
        'BvnNo')['Amount'].sum().rename('latest_salary_payment_amount_last_3_months').reset_index()

    last_3_months_salaries_summaries = pd.merge(last_3_months_salaries_summaries, monthly_salary_summaries, on='BvnNo')

    last_3_months_salaries_summaries = pd.merge(last_3_months_salaries_summaries,
                                                latest_salary_payment_amount_last_3_months, on='BvnNo')

    last_3_months_salaries_summaries['is_qualified_on_salaries'] = last_3_months_salaries_summaries.apply(
        lambda x: calculate_last_3_months_salaries_government(x), axis=1)

    return last_3_months_salaries_summaries

In [18]:
def calculate_last_3_months_salaries_government(df):

    count = df['count_of_payments_last_3_months']

    if count >= 2 :
        return True
    else:
        return False

In [19]:
salaries_df_1 = government_employees_salaries_check_30_day_product(salaries, max_month)

salaries_df_1.head(2)

/apps/conda/fiona.kamau/envs/tanda/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,BvnNo,count_of_payments_last_3_months,sum_of_salary_payments_last_3_months,minimum_salary_payment_last_3_months,earliest_salary_payment_date_last_3_months,latest_salary_payment_date_last_3_months,average_monthly_salary_last_3_months,latest_salary_payment_amount_last_3_months,is_qualified_on_salaries
0,22159706127,1,251130.91,251130.91,2023-05-02,2023-05-02,251130.91,251130.91,False
1,22312111678,1,142902.99,4191.17,2023-05-02,2023-05-02,142902.99,142902.99,False


In [ ]:
def non_government_employees_salaries_check_3_and_6_month_product(salaries, max_month):
    last_6_months = salaries[salaries['offset_payment_dates'] >= max_month + relativedelta(
        months=-5)]  # & (salaries['payment_dates'] < max_month)]

    monthly_salary_summaries = last_6_months.groupby(['BvnNo', 'year_month_salary_date'])['Amount'].sum().reset_index()

    monthly_salary_summaries = monthly_salary_summaries.groupby('BvnNo')['Amount'].mean().rename(
        'average_monthly_salary_last_6_months').reset_index()

    last_6_months_salaries_summaries = last_6_months.groupby(['BvnNo']).agg(
        count_of_payments_last_6_months=pd.NamedAgg('year_month_salary_date', 'nunique'),
        sum_of_salary_payments_last_6_months=pd.NamedAgg('Amount', 'sum'),
        minimum_salary_payment_last_6_months=pd.NamedAgg('Amount', 'min'),
        # median_salary_payment_last_6_months = pd.NamedAgg('Amount', 'median'),
        # mode_salary_payment_last_6_months = pd.NamedAgg('Amount', pd.Series.mode),
        earliest_salary_payment_date_last_6_months=pd.NamedAgg('payment_dates', 'min'),
        latest_salary_payment_date_last_6_months=pd.NamedAgg('payment_dates', 'max'),
        ).reset_index()

    latest_payment_dates = last_6_months_salaries_summaries[['BvnNo', 'latest_salary_payment_date_last_6_months']]

    latest_payment_dates.loc[:, 'year_month_latest_salary_date'] = [x.strftime('%Y-%m') for x in latest_payment_dates[
        'latest_salary_payment_date_last_6_months']]

    last_6_months = last_6_months.merge(latest_payment_dates, on='BvnNo')
    latest_salary_payment_amount_last_6_months = \
    last_6_months[last_6_months['year_month_latest_salary_date'] == last_6_months['year_month_salary_date']].groupby(
        'BvnNo')['Amount'].sum().rename('latest_salary_payment_amount_last_6_months').reset_index()

    last_6_months_salaries_summaries = pd.merge(last_6_months_salaries_summaries, monthly_salary_summaries, on='BvnNo')

    last_6_months_salaries_summaries = pd.merge(last_6_months_salaries_summaries,
                                                latest_salary_payment_amount_last_6_months, on='BvnNo')

    last_6_months_salaries_summaries['is_qualified_on_salaries'] = last_6_months_salaries_summaries.apply(
        lambda x: calculate_last_6_months_salaries(x), axis=1)

    return last_6_months_salaries_summaries

In [13]:
def government_employees_salaries_check_3_and_6_month_product(salaries, max_month):
    last_9_months = salaries[salaries['offset_payment_dates'] >= max_month + relativedelta(
        months=-8)]  # & (salaries['payment_dates'] < max_month)]

    monthly_salary_summaries = last_9_months.groupby(['BvnNo', 'year_month_salary_date'])['Amount'].sum().reset_index()

    monthly_salary_summaries = monthly_salary_summaries.groupby('BvnNo')['Amount'].mean().rename(
        'average_monthly_salary_last_9_months').reset_index()

    last_9_months_salaries_summaries = last_9_months.groupby(['BvnNo']).agg(
        count_of_payments_last_9_months=pd.NamedAgg('year_month_salary_date', 'nunique'),
        sum_of_salary_payments_last_9_months=pd.NamedAgg('Amount', 'sum'),
        minimum_salary_payment_last_9_months=pd.NamedAgg('Amount', 'min'),
        # median_salary_payment_last_6_months = pd.NamedAgg('Amount', 'median'),
        # mode_salary_payment_last_6_months = pd.NamedAgg('Amount', pd.Series.mode),
        earliest_salary_payment_date_last_9_months=pd.NamedAgg('payment_dates', 'min'),
        latest_salary_payment_date_last_9_months=pd.NamedAgg('payment_dates', 'max')
        ).reset_index()

    latest_payment_dates = last_9_months_salaries_summaries[['BvnNo', 'latest_salary_payment_date_last_9_months']]

    latest_payment_dates.loc[:, 'year_month_latest_salary_date'] = [x.strftime('%Y-%m') for x in latest_payment_dates[
        'latest_salary_payment_date_last_9_months']]

    last_9_months = last_9_months.merge(latest_payment_dates, on = 'BvnNo')

    latest_salary_payment_amount_last_9_months = last_9_months[last_9_months['year_month_latest_salary_date'] == last_9_months['year_month_salary_date']].groupby(
        'BvnNo')['Amount'].sum().rename('latest_salary_payment_amount_last_9_months').reset_index()

    last_9_months_salaries_summaries = pd.merge(last_9_months_salaries_summaries, monthly_salary_summaries, on='BvnNo')

    last_9_months_salaries_summaries = pd.merge(last_9_months_salaries_summaries,
                                                latest_salary_payment_amount_last_9_months, on='BvnNo')

    last_9_months_salaries_summaries['is_qualified_on_salaries'] = last_9_months_salaries_summaries.apply(
        lambda x: calculate_last_9_months_salaries(x), axis=1)

    return last_9_months_salaries_summaries

In [ ]:
def calculate_last_6_months_salaries(df):

    count = df['count_of_payments_last_6_months']
    #company_name = df['CompanyName']
    average_salary = df['average_monthly_salary_last_6_months']
    sum_salary = df['sum_of_salary_payments_last_6_months']


    if count >= 6 :
        return True
    else:
        total_salary = (average_salary * 6)
        if sum_salary >= total_salary: ## if received >= simulation --> True
            return True
        else:
            return False

In [ ]:
def non_government_employees_salaries_check_30_day_product(salaries, max_month):
    last_3_months = salaries[salaries['offset_payment_dates'] >= max_month + relativedelta(
        months=-2)]  # & (salaries['payment_dates'] < max_month)]

    monthly_salary_summaries = last_3_months.groupby(['BvnNo', 'year_month_salary_date'])['Amount'].sum().reset_index()

    monthly_salary_summaries = monthly_salary_summaries.groupby('BvnNo')['Amount'].mean().rename(
        'average_monthly_salary_last_3_months').reset_index()

    last_3_months_salaries_summaries = last_3_months.groupby(['BvnNo']).agg(
        count_of_payments_last_3_months=pd.NamedAgg('year_month_salary_date', 'nunique'),
        sum_of_salary_payments_last_3_months=pd.NamedAgg('Amount', 'sum'),
        minimum_salary_payment_last_3_months=pd.NamedAgg('Amount', 'min'),
        # median_salary_payment_last_3_months = pd.NamedAgg('Amount', 'median'),
        # mode_salary_payment_last_3_months = pd.NamedAgg('Amount', pd.Series.mode),
        earliest_salary_payment_date_last_3_months=pd.NamedAgg('payment_dates', 'min'),
        latest_salary_payment_date_last_3_months=pd.NamedAgg('payment_dates', 'max'),
        ).reset_index()

    latest_payment_dates = last_3_months_salaries_summaries[['BvnNo', 'latest_salary_payment_date_last_3_months']]

    latest_payment_dates.loc[:, 'year_month_latest_salary_date'] = [x.strftime('%Y-%m') for x in latest_payment_dates[
        'latest_salary_payment_date_last_3_months']]

    last_3_months = last_3_months.merge(latest_payment_dates, on='BvnNo')
    latest_salary_payment_amount_last_3_months = \
    last_3_months[last_3_months['year_month_latest_salary_date'] == last_3_months['year_month_salary_date']].groupby(
        'BvnNo')['Amount'].sum().rename('latest_salary_payment_amount_last_3_months').reset_index()

    last_3_months_salaries_summaries = pd.merge(last_3_months_salaries_summaries, monthly_salary_summaries, on='BvnNo')

    last_3_months_salaries_summaries = pd.merge(last_3_months_salaries_summaries,
                                                latest_salary_payment_amount_last_3_months, on='BvnNo')

    last_3_months_salaries_summaries['is_qualified_on_salaries'] = last_3_months_salaries_summaries.apply(
        lambda x: calculate_last_3_months_salaries_non_government(x), axis=1)

    return last_3_months_salaries_summaries

In [ ]:
def calculate_last_3_months_salaries_non_government(df):

    count = df['count_of_payments_last_3_months']
    average_salary = df['average_monthly_salary_last_3_months']
    sum_salary = df['sum_of_salary_payments_last_3_months']


    if count >= 2 :
        return True
    else:
        total_salary = (average_salary * 3)
        if sum_salary >= total_salary: ## if received >= simulation --> True
            return True
        else:
            return False

### DTI

In [ ]:
def calculate_additional_summaries(salaries_summaries: pd.DataFrame, loans: pd.DataFrame, time_period: int) -> pd.DataFrame:
    
    monthly_expected_repayments = loans.groupby('BvnNo')['RepaymentAmount'].sum().rename(
        'total_debt_value').reset_index()
    summaries = pd.merge(salaries_summaries, monthly_expected_repayments, on='BvnNo', how='left')

    summaries['total_debt_value'].fillna(0, inplace=True)

    summaries['DTI'] = round(
        summaries['total_debt_value'] / summaries[f'average_monthly_salary_last_{time_period}_months'], 1)
    summaries['DTI'].fillna(1, inplace=True) ## update to 0
    
    
    
    
    active_loans = loans[loans['OutstandingAmount'] > 0]
    all_running_loans = active_loans.groupby('BvnNo')['disbursement_dates'].nunique().rename(
        'count_of_all_running_loans').reset_index()

    summaries = pd.merge(summaries, all_running_loans, on='BvnNo', how='outer')
    summaries['count_of_all_running_loans'].fillna(0, inplace=True)

    
    
    summaries['latest_sal_ge_min_payment'] = summaries[f'latest_salary_payment_amount_last_{time_period}_months'] >= \
                                             summaries[
                                                 f'minimum_salary_payment_last_{time_period}_months']

#### CRC Checks

In [ ]:
## 3 and 6 month product

final_df.loc[(final_df['total_no_of_running_facilities'] <= 5) &
                 (final_df['maximum_days_in_arrears'] < 60) &
                 (final_df['total_overdue_amount'] < scoring_summaries['average_salary_payment_last_9_months']) & ## last 9 or 6 months depending on if gvmnt ot not
                 (final_df['no_of_institution_types'] <= 2), 'is_crb_qualified'] = True

In [ ]:
## 30 day product  - government employee
 
final_df.loc[(final_df['total_no_of_running_facilities'] <= 5) &
                 (final_df['maximum_days_in_arrears'] < 90) &
                 (final_df['total_overdue_amount'] < scoring_summaries['average_salary_payment_last_3_months']) &
                 (scoring_summaries['is_gvnt_employee'] == True) &
                 (final_df['no_of_institution_types'] <= 2), 'is_crb_qualified'] = True

In [ ]:
## 30 day product - non-government employee
 
final_df.loc[(final_df['total_no_of_running_facilities'] <= 5) &
                 (final_df['maximum_days_in_arrears'] < 60) &
                 (final_df['total_overdue_amount'] < scoring_summaries['average_salary_payment_last_3_months']) &
                 (scoring_summaries['is_gvnt_employee'] == False) &
                 (final_df['no_of_institution_types'] <= 2), 'is_crb_qualified'] = True

In [ ]:
## update tenures to be shared to include new product